In [10]:
import numpy as np
part1 = np.load("part1a.npz")
part1.files
Ic_0 = part1["Ic_0"]
N = part1["N"]
gamma = part1["gamma"]
Lc = part1["Lc"]
Svc_0 = part1["Svc_0_pmf"]

In [11]:
import pandas as pd
S_0 = N-(Ic_0.sum())
I_0 = Ic_0.sum()
R_0 = 0
data = pd.DataFrame(Svc_0)
data["Ic_0"] = Ic_0
data

,0,1,2,3,Ic_0
0,0.109533,0.033008,0.081790,0.115940,47
1,0.030898,0.085120,0.045288,0.057183,90
2,0.074715,0.024956,0.102366,0.042567,51
3,0.016059,0.081096,0.091582,0.007900,65


In [12]:
import random 
from random import Random
random.seed(a=None, version=2)
B0c = []
B1c = []
B2c = []
B3c = []
for i in range(4):
    B0c.append(random.uniform(0, 0.25))
    B1c.append(random.uniform(0.25, 0.5))
    B2c.append(random.uniform(0.5, 0.75))
    B3c.append(random.uniform(0.75, 1.0))

In [13]:
Bvc = np.array([B0c, B1c, B2c, B3c])
for i in range(4):
    Bvc[i].sort()
Bvc = pd.DataFrame(Bvc)
Bvc

,0,1,2,3
0,0.004949,0.083789,0.089729,0.209132
1,0.276294,0.295756,0.366533,0.440478
2,0.685224,0.709078,0.724805,0.749249
3,0.784072,0.834567,0.843017,0.858517


In [14]:
def S_rate(Svc, I, N, Bvc, gamma):
    """Calculate the rate of change of suceptible group split by vulnerability and comorbidity, return a
    tuple where the first element is split on both categories, and the second element only split by comorbidity"""
    #create empty data frame to store results
    dSvcdt = pd.DataFrame()
    #nested for loop, first level iterates through vulnerability
    forloop_start = time.time_ns()
    for v in range(4):
        #create an empty list for each comorbidity level
        dScdt = []
        for c in range(4):
            #here we calculate the value of dSdt for each comorbidity group in this iteration
            dScdt.append(-(Bvc[v][c] * (Svc[v][c]) * I) / N)
        #assign the list as a column in the data frame, at index c.
        dSvcdt[v] = dScdt
    forloop_end = time.time_ns()
    print("forloop", forloop_end - forloop_start/1000)
    sumstart = time.time_ns()
    dSdt = dSvcdt.sum(axis = 0)
    sumend = time.time_ns()
    print("sum", sumend - sumstart/1000)
    return dSvcdt, dSdt

In [20]:
def I_rate(dSdt, Ic, gamma):
    """This function takes in the rate of change of S split by comorbidity, I split by comorbidity and gamma.
    It returns a tuple, the first element is dIdt split by comorbidity, the second element is the aggregate"""
    dIcdt = []
    dIdt = 0
    for c in range(4):
        #now we implement the rest of the formula
        dIcdt.append(-dSdt[c] - gamma*Ic[c])
        dIdt += -dSdt[c] - gamma*Ic[c]
    return dIcdt, dIdt

SyntaxError: invalid syntax (<ipython-input-20-ee9128a33741>, line 11)

In [16]:
def R_rate(dIdt, gamma):
    dRdt = gamma * dIdt[1]
    return dRdt

In [18]:
import time
Svc = pd.DataFrame(Svc_0*N)
Ic = Ic_0
I = I_0
S = N - I
R = 0
simulation = [[S, I, R]]
for i in range(20):
    dSdt_start = time.time_ns()
    dSdt = S_rate(Svc, Ic, N, Bvc, gamma)
    #print(dSdt)
    dSdt_end = time.time_ns()
    print((dSdt_end-dSdt_start)/1000)
    dIdt = I_rate(dSdt[1], Ic, gamma)
    #print(dIdt)
    dRdt_start = time.time_ns()
    print((dSdt_end - dRdt_start)/1000)
    dRdt = R_rate(dIdt[1], gamma)
    #print(dRdt)
    dRdt_end = time.time_ns()
    print((dRdt_end - dRdt_start)/1000)
    Svc = Svc + dSdt[0]
    S = S + dSdt[1].sum()
    #print(S)
    Ic = Ic + dIdt[0]
    I = I + dIdt[1].sum()
    #print(I)
    R = R + dRdt
    simulation.append([[S, I, R]])
simulation

forloop 1.6013623684859868e+18
sum 1.6013623685230328e+18
211553.2
-62485.8
0.0
forloop 1.6013623687644815e+18
sum 1.6013623687654623e+18
20600.2
0.0
0.0
forloop 1.6013623687694456e+18
sum 1.6013623687714368e+18
4984.8
0.0
0.0
forloop 1.6013623687764232e+18
sum 1.6013623687774172e+18
4987.1
0.0
0.0
forloop 1.6013623687863862e+18
sum 1.6013623687873795e+18
4986.6
-997.7
0.0
forloop 1.6013623687943567e+18
sum 1.601362368796347e+18
6982.2
-999.4
0.0
forloop 1.6013623688023276e+18
sum 1.601362368804318e+18
5983.9
-996.8
0.0
forloop 1.6013623688162806e+18
sum 1.6013623688202614e+18
11968.5
-3058.8
0.0
forloop 1.601362368854562e+18
sum 1.6013623688701553e+18
46872.3
-15617.2
15622.0
forloop 1.601362369121412e+18
sum 1.6013623691764419e+18
262683.8
-75797.2
998.1
forloop 1.60136237016726e+18
sum 1.6013623703828183e+18
1032309.2
-322208.3
0.0
forloop 1.6013623864314688e+18
sum 1.6013623918424402e+18
20395390.2


KeyboardInterrupt: 

In [9]:
simulation = pd.DataFrame(np.array([[S_0, I, R]]), columns = ["S", "I", "R"])

<ipython-input-9-61ad8cb34b27>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  simulation = pd.DataFrame(np.array([[S_0, I, R]]), columns = ["S", "I", "R"])


In [ ]:
simulation.append(["1", "2","3"])
simulation